# Trabalho NLP - Desafio 01

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import re
import numpy as np
from time import time
from pprint import pprint

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

nltk.download('stopwords')
nltk.download('punkt')


from nltk.stem import SnowballStemmer

from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#modelos
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mbjesus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mbjesus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Etapa 1 - Obter dados Rotulados

In [2]:
dataset = pd.read_csv("dataset/train.csv")
datasetTeste = pd.read_csv("dataset/test.csv")

In [3]:
datasetTeste["id"] = datasetTeste.index
datasetTeste.head(10)


,title,id
0,Strobo Mini Kit Com 5 Strobos,0
1,Filtro Hoya Hmc Uv(c) - Uv-hmc 72mm,1
2,Dvd Next Lite (envio Toda Segunda),2
3,Fim De Curso Tubinho Do Motor Desl Peccinin ...,3
4,Carro Infantil Smart Baby Reclinável Bandeiran...,4
5,Par Coifa Caixa Direção Hidraulica C4 Lounge 2...,5
6,Protetor Sofá Retrátil Reclinavél Com Assento ...,6
7,Pedal Behringer Tu300 Afinador Cromatico,7
8,Receptor Satmax Elsys,8
9,Mini Fantasia Noivinha,9


## Etapa 2 - Pré-processamento 

### 2.1 - Converter para minúsculo

In [4]:
dataset["category"] = dataset["category"].str.lower()
dataset["title"] = dataset["title"].str.lower()
dataset.head(5)


,title,category
0,tampa para champanhe cromado ibili - 722500,ice_buckets
1,copo térmico plástico os jetsons família espacial,thermal_cups_and_tumblers
2,nobreak sms bivolt 600va para pc ps4 xbox,stabilizers_and_ups
3,led de balada rgb luz de palco dj disco strobo...,led_stage_lights
4,filtro de lente cpl 52mm kenko polarizador par...,lens_filters


In [5]:
datasetTeste["title"] = datasetTeste["title"].str.lower()
datasetTeste.head(5)

,title,id
0,strobo mini kit com 5 strobos,0
1,filtro hoya hmc uv(c) - uv-hmc 72mm,1
2,dvd next lite (envio toda segunda),2
3,fim de curso tubinho do motor desl peccinin ...,3
4,carro infantil smart baby reclinável bandeiran...,4


### 2.2 - Remover caractéres especiais

In [48]:
##FUNCAO QUE REMOVE CARACTERES ESPECIAIS
def remover_acentos(texto):
    
    ##REMOVER ACENTOS
    #texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    
    ##REMOVER CARACTERES ESPECIAIS
    texto = re.sub(r'[?|!|$|&|*|%|@|(|)|~|,|\.|:|\-|\'|/|+]', r' ', texto)
    
    ##REMOVER MEDIDAS
    texto = re.sub('(va)|(mm)|(km)|(cm)', r' ', texto)
    
    ##REMOVER NUMEROS QUE APARECEM ISOLADOS
    texto = re.sub('[\s][0-9]{1,}', ' ', texto)
    
     ##REMOVER TEXTOS COM ATE DOIS CARACTERES
    texto = re.sub(r'\b\w{1,2}\b', ' ', texto)
    
    return texto



In [49]:
#aplica limpeza no texto
dataset["title_normalizado"] = dataset["title"].map(remover_acentos)
dataset.head(5)


,title,category,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
0,tampa para champanhe cromado ibili - 722500,ice_buckets,tampa para champanhe cromado ibili,tampa champanhe cromado ibili,tamp champanh crom ibil
1,copo térmico plástico os jetsons família espacial,thermal_cups_and_tumblers,copo térmico plástico jetsons família espacial,copo termico plastico jetsons familia espacial,cop termic plastic jetsons famil espacial
2,nobreak sms bivolt 600va para pc ps4 xbox,stabilizers_and_ups,nobreak sms bivolt para ps4 xbox,nobreak sms bivolt ps4 xbox,nobreak sms bivolt ps4 xbox
3,led de balada rgb luz de palco dj disco strobo...,led_stage_lights,led balada rgb luz palco disco strobo fe...,led balada rgb luz palco disco strobo festa ba...,led bal rgb luz palc disc strob fest bal
4,filtro de lente cpl 52mm kenko polarizador par...,lens_filters,filtro lente cpl kenko polarizador para ob...,filtro lente cpl kenko polarizador objeti,filtr lent cpl kenk polariz objet


In [50]:
datasetTeste["title_normalizado"] = datasetTeste["title"].map(remover_acentos)
datasetTeste.head(5)

,title,id,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
0,strobo mini kit com 5 strobos,0,strobo mini kit com strobos,strobo mini kit strobos,strob min kit strob
1,filtro hoya hmc uv(c) - uv-hmc 72mm,1,filtro hoya hmc hmc,filtro hoya hmc hmc,filtr hoy hmc hmc
2,dvd next lite (envio toda segunda),2,dvd next lite envio toda segunda,dvd next lite envio toda segunda,dvd next lit envi tod segund
3,fim de curso tubinho do motor desl peccinin ...,3,fim curso tubinho motor desl peccinin li...,fim curso tubinho motor desl peccinin light super,fim curs tubinh motor desl peccinin light sup
4,carro infantil smart baby reclinável bandeiran...,4,carro infantil smart baby reclinável bandeiran...,carro infantil smart baby reclinavel bandeiran...,carr infantil smart baby reclinavel bandeir ros


### 2.3 - Remover Stop-words

In [51]:
def remove_stopwords(texto):
    #REMOVER STOPWORDS
    ##baixar stopwords
    stopwords = nltk.corpus.stopwords.words('portuguese')
    
    #gera tokens
    tokens = word_tokenize(texto)
    
    #remove stopwords
    filtered_tokens = [token.lower() for token in tokens if token not in stopwords]
    
    #retorna texto sem as stop-words
    return ' '.join(filtered_tokens)

In [52]:
#aplica remocao de stop-words
dataset["title_normalizado_sem_sw"] = dataset["title_normalizado"].map(remove_stopwords)
dataset.head(5)

,title,category,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
0,tampa para champanhe cromado ibili - 722500,ice_buckets,tampa para champanhe cromado ibili,tampa champanhe cromado ibili,tamp champanh crom ibil
1,copo térmico plástico os jetsons família espacial,thermal_cups_and_tumblers,copo térmico plástico jetsons família espacial,copo térmico plástico jetsons família espacial,cop termic plastic jetsons famil espacial
2,nobreak sms bivolt 600va para pc ps4 xbox,stabilizers_and_ups,nobreak sms bivolt para ps4 xbox,nobreak sms bivolt ps4 xbox,nobreak sms bivolt ps4 xbox
3,led de balada rgb luz de palco dj disco strobo...,led_stage_lights,led balada rgb luz palco disco strobo fe...,led balada rgb luz palco disco strobo festa ba...,led bal rgb luz palc disc strob fest bal
4,filtro de lente cpl 52mm kenko polarizador par...,lens_filters,filtro lente cpl kenko polarizador para ob...,filtro lente cpl kenko polarizador objeti,filtr lent cpl kenk polariz objet


In [53]:
datasetTeste["title_normalizado_sem_sw"] = datasetTeste["title_normalizado"].map(remove_stopwords)
datasetTeste.head(5)

,title,id,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
0,strobo mini kit com 5 strobos,0,strobo mini kit com strobos,strobo mini kit strobos,strob min kit strob
1,filtro hoya hmc uv(c) - uv-hmc 72mm,1,filtro hoya hmc hmc,filtro hoya hmc hmc,filtr hoy hmc hmc
2,dvd next lite (envio toda segunda),2,dvd next lite envio toda segunda,dvd next lite envio toda segunda,dvd next lit envi tod segund
3,fim de curso tubinho do motor desl peccinin ...,3,fim curso tubinho motor desl peccinin li...,fim curso tubinho motor desl peccinin light super,fim curs tubinh motor desl peccinin light sup
4,carro infantil smart baby reclinável bandeiran...,4,carro infantil smart baby reclinável bandeiran...,carro infantil smart baby reclinável bandeiran...,carr infantil smart baby reclinavel bandeir ros


### 2.4 - Transformar para Radical

In [54]:
def stemmer_sentenca(texto):
    
    #baixa dados
    pt_stemmer = SnowballStemmer('portuguese')
    
    #divide em tokens
    tokens = word_tokenize(texto)
    
    #aplica stemmer
    lista_stem = [pt_stemmer.stem(token) for token in tokens ]
    
    #retorna texto 
    return ' '.join(lista_stem)

In [55]:
#aplica limpeza stemmer
dataset["title_normalizado_sem_sw_stemmer"] = dataset["title_normalizado_sem_sw"].map(stemmer_sentenca)
dataset.head(5)

,title,category,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
0,tampa para champanhe cromado ibili - 722500,ice_buckets,tampa para champanhe cromado ibili,tampa champanhe cromado ibili,tamp champanh crom ibil
1,copo térmico plástico os jetsons família espacial,thermal_cups_and_tumblers,copo térmico plástico jetsons família espacial,copo térmico plástico jetsons família espacial,cop térmic plástic jetsons famíl espacial
2,nobreak sms bivolt 600va para pc ps4 xbox,stabilizers_and_ups,nobreak sms bivolt para ps4 xbox,nobreak sms bivolt ps4 xbox,nobreak sms bivolt ps4 xbox
3,led de balada rgb luz de palco dj disco strobo...,led_stage_lights,led balada rgb luz palco disco strobo fe...,led balada rgb luz palco disco strobo festa ba...,led bal rgb luz palc disc strob fest bal
4,filtro de lente cpl 52mm kenko polarizador par...,lens_filters,filtro lente cpl kenko polarizador para ob...,filtro lente cpl kenko polarizador objeti,filtr lent cpl kenk polariz objet


In [56]:
datasetTeste["title_normalizado_sem_sw_stemmer"] = datasetTeste["title_normalizado_sem_sw"].map(stemmer_sentenca)
datasetTeste.head(5)

,title,id,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
0,strobo mini kit com 5 strobos,0,strobo mini kit com strobos,strobo mini kit strobos,strob min kit strob
1,filtro hoya hmc uv(c) - uv-hmc 72mm,1,filtro hoya hmc hmc,filtro hoya hmc hmc,filtr hoy hmc hmc
2,dvd next lite (envio toda segunda),2,dvd next lite envio toda segunda,dvd next lite envio toda segunda,dvd next lit envi tod segund
3,fim de curso tubinho do motor desl peccinin ...,3,fim curso tubinho motor desl peccinin li...,fim curso tubinho motor desl peccinin light super,fim curs tubinh motor desl peccinin light sup
4,carro infantil smart baby reclinável bandeiran...,4,carro infantil smart baby reclinável bandeiran...,carro infantil smart baby reclinável bandeiran...,carr infantil smart baby reclin bandeir ros


### 2.5 - Balancear Dados com Uper-Undersampling

In [57]:
def balencearMesmaClasse(df_desbalanceado,tamanho):
        
    lista_categorias = df_desbalanceado.category.unique()
        
    df_sampled = pd.DataFrame()

    for categoria in lista_categorias:

        df_tmp = df_desbalanceado[df_desbalanceado.category == categoria]

        if(len(df_tmp)) > tamanho:
            ##como a classe tem mais registros que o tamanho, fazer SEM repeticao
            df_tmp = resample(df_tmp, 
                             replace=False,    
                             n_samples=(tamanho),    
                             random_state=123) 
        else:
            ##como a classe tem mais registros que o tamanho, fazer COM repeticao
            df_tmp = resample(df_tmp, 
                             replace=True,    
                             n_samples=(tamanho),    
                             random_state=123)

        df_sampled = pd.concat([df_sampled, df_tmp])

    return df_sampled

In [58]:
dataset_balanceado = balencearMesmaClasse(dataset,370)
dataset_balanceado.head(5)

,title,category,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
20135,balde de gelo acrílico p/ cerveja 5l baldinho ...,ice_buckets,balde gelo acrílico cerveja baldinho ge...,balde gelo acrílico cerveja baldinho gelo latas,bald gel acríl cervej baldinh gel lat
3343,cervegela c/ refil de congelamento 2h mantém c...,ice_buckets,cervegela refil congelamento mantém cer...,cervegela refil congelamento mantém cerveja ge...,cervegel refil congel mantém cervej gel
24862,balde brahma com alça e abridor de garrafas,ice_buckets,balde brahma com alça abridor garrafas,balde brahma alça abridor garrafas,bald brahm alça abridor garraf
14064,kit c/ 6 balde plástico reforçado plasnew 20 l...,ice_buckets,kit balde plástico reforçado plasnew litros,kit balde plástico reforçado plasnew litros,kit bald plástic reforc plasnew litr
27396,**balde brahma - luminoso de brinde!!!!,ice_buckets,balde brahma luminoso brinde,balde brahma luminoso brinde,bald brahm lumin brind


In [59]:
dataset_balanceado.shape

(30710, 5)

In [60]:
dataset.shape

(33500, 5)

In [61]:
###Separar os dados em treino e teste """
X_train, X_test, y_train, y_test = train_test_split(dataset_balanceado["title_normalizado_sem_sw_stemmer"], 
                                                    dataset_balanceado["category"], 
                                                    test_size=0.3, random_state = 0)

#CONSTRUIR MATRIZ ESPARSA """ 
count_vect = CountVectorizer(max_df= 0.5,
                            max_features= None,
                            min_df= 1,
                            ngram_range= (1, 2))

#fazer fit_transoform (vetor de palavras) para os dados de treino """
X_train_counts = count_vect.fit_transform(X_train)

#ponderar palavras mais frequentes  """
tf_transformer = TfidfTransformer(norm= 'l2',
                                  use_idf= True)
X_train_tf = tf_transformer.fit_transform(X_train_counts)

'''
Melhor score: 0.956
Melhores Parametros:
	clf__alpha: 1e-05
	clf__max_iter: 80
	clf__penalty: 'l2'
	tfidf__norm: 'l2'
	tfidf__use_idf: True
	vect__max_df: 0.5
	vect__max_features: None
	vect__min_df: 1
	vect__ngram_range: (1, 2)
       ''' 

"\nMelhor score: 0.956\nMelhores Parametros:\n\tclf__alpha: 1e-05\n\tclf__max_iter: 80\n\tclf__penalty: 'l2'\n\ttfidf__norm: 'l2'\n\ttfidf__use_idf: True\n\tvect__max_df: 0.5\n\tvect__max_features: None\n\tvect__min_df: 1\n\tvect__ngram_range: (1, 2)\n       "

## Etapa 3 - Aplicar aprendizado de máquina 

In [62]:
sgd = SGDClassifier(penalty='l2',
                    alpha=1e-05, 
                    max_iter=80)



#treinar modelo
t0 = time()
clf = sgd.fit(X_train_tf, y_train)
print("Fit done in %0.3fs" % (time() - t0))
y_pred_modelo = clf.predict(count_vect.transform(X_test))
print("Pred done in %0.3fs" % (time() - t0))




Fit done in 1.101s
Pred done in 1.249s


### Calcular desempenho 

In [92]:
saida = classification_report(y_test, y_pred_modelo, target_names=dataset_balanceado["category"].unique(),output_dict=True)
dfResultado = pd.DataFrame(saida).transpose()
df

,f1-score,precision,recall,support
ice_buckets,0.903226,0.954545,0.857143,98.000000
thermal_cups_and_tumblers,0.937799,0.933333,0.942308,104.000000
stabilizers_and_ups,0.927928,0.919643,0.936364,110.000000
led_stage_lights,0.930041,0.918699,0.941667,120.000000
lens_filters,0.990476,0.990476,0.990476,105.000000
engine_oil_dipsticks,0.928230,0.932692,0.923810,105.000000
roller_skates,0.973214,0.981982,0.964602,113.000000
pool_lights,0.916667,0.933962,0.900000,110.000000
key_racks,0.889831,0.921053,0.860656,122.000000
tv_antennas,0.897778,0.885965,0.909910,111.000000


In [93]:
## Mostar menores desempenhos
df[df["precision"] < 0.90 ]

,f1-score,precision,recall,support
tv_antennas,0.897778,0.885965,0.909910,111.0
musical_tuners,0.940000,0.895238,0.989474,95.0
cellphone_repair_tool_kits,0.932735,0.888889,0.981132,106.0
lighters,0.876712,0.872727,0.880734,109.0
flutes,0.828829,0.793103,0.867925,106.0
sofa_and_futon_covers,0.890830,0.836066,0.953271,107.0
push_and_riding_toys,0.914573,0.883495,0.947917,96.0


In [94]:
##Acuracia global
accuracy_score(y_test, y_pred_modelo)


0.950070552480191

# Aplicar modelo nos dados de Teste

In [65]:
##Visao dos dados de Teste
datasetTeste.head(5)


,title,id,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
0,strobo mini kit com 5 strobos,0,strobo mini kit com strobos,strobo mini kit strobos,strob min kit strob
1,filtro hoya hmc uv(c) - uv-hmc 72mm,1,filtro hoya hmc hmc,filtro hoya hmc hmc,filtr hoy hmc hmc
2,dvd next lite (envio toda segunda),2,dvd next lite envio toda segunda,dvd next lite envio toda segunda,dvd next lit envi tod segund
3,fim de curso tubinho do motor desl peccinin ...,3,fim curso tubinho motor desl peccinin li...,fim curso tubinho motor desl peccinin light super,fim curs tubinh motor desl peccinin light sup
4,carro infantil smart baby reclinável bandeiran...,4,carro infantil smart baby reclinável bandeiran...,carro infantil smart baby reclinável bandeiran...,carr infantil smart baby reclin bandeir ros


### Aplicando predição

In [66]:

y_pred_modelo_final = clf.predict(count_vect.transform(datasetTeste["title_normalizado_sem_sw_stemmer"]))


In [67]:
data_setPredito = pd.DataFrame(y_pred_modelo_final,columns=["category"])
data_setPredito.head(10)

,category
0,led_stage_lights
1,lens_filters
2,cd_and_dvd_players
3,gate_motors
4,scooters
5,power_steering_hoses
6,sofa_and_futon_covers
7,musical_tuners
8,tv_receivers_and_decoders
9,erotic_costumes


In [68]:
##Transformar em Data-Frame
data_setPredito['id'] = data_setPredito.index
data_setPredito['category'] = data_setPredito['category'].str.upper()
submissao = data_setPredito[["id","category"]] 

## Salvar no formato de submissao 
submissao.to_csv("submissao.csv", index = None, header=True)
submissao.head(10)


,id,category
0,0,LED_STAGE_LIGHTS
1,1,LENS_FILTERS
2,2,CD_AND_DVD_PLAYERS
3,3,GATE_MOTORS
4,4,SCOOTERS
5,5,POWER_STEERING_HOSES
6,6,SOFA_AND_FUTON_COVERS
7,7,MUSICAL_TUNERS
8,8,TV_RECEIVERS_AND_DECODERS
9,9,EROTIC_COSTUMES


### Gerando visualizacão amigável

In [71]:
visualizacaoResutado =   pd.merge(datasetTeste[["title","title_normalizado_sem_sw","title_normalizado_sem_sw_stemmer","id"]],
                         data_setPredito[["id","category"]],
                         on='id')
visualizacaoResutado.head(200)


,title,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer,id,category
0,strobo mini kit com 5 strobos,strobo mini kit strobos,strob min kit strob,0,LED_STAGE_LIGHTS
1,filtro hoya hmc uv(c) - uv-hmc 72mm,filtro hoya hmc hmc,filtr hoy hmc hmc,1,LENS_FILTERS
2,dvd next lite (envio toda segunda),dvd next lite envio toda segunda,dvd next lit envi tod segund,2,CD_AND_DVD_PLAYERS
3,fim de curso tubinho do motor desl peccinin ...,fim curso tubinho motor desl peccinin light super,fim curs tubinh motor desl peccinin light sup,3,GATE_MOTORS
4,carro infantil smart baby reclinável bandeiran...,carro infantil smart baby reclinável bandeiran...,carr infantil smart baby reclin bandeir ros,4,SCOOTERS
5,par coifa caixa direção hidraulica c4 lounge 2...,par coifa caixa direção hidraulica lounge,par coif caix direçã hidraul loung,5,POWER_STEERING_HOSES
6,protetor sofá retrátil reclinavél com assento ...,protetor sofá retrátil reclinavél assento módulos,protetor sof retrátil reclinavél assent módul,6,SOFA_AND_FUTON_COVERS
7,pedal behringer tu300 afinador cromatico,pedal behringer tu300 afinador cromatico,pedal behring tu300 afin cromat,7,MUSICAL_TUNERS
8,receptor satmax elsys,receptor satmax elsys,receptor satmax elsys,8,TV_RECEIVERS_AND_DECODERS
9,mini fantasia noivinha,mini fantasia noivinha,min fantas noivinh,9,EROTIC_COSTUMES


# FIM 

In [70]:
dataset_balanceado[ dataset_balanceado["category"] == 'flutes']

,title,category,title_normalizado,title_normalizado_sem_sw,title_normalizado_sem_sw_stemmer
10070,"flauta transversal king , americana prateada,...",flutes,flauta transversal king americana prateada ...,flauta transversal king americana prateada semi,flaut transversal king american prat sem
24009,flauta slade transversal prata ccb em dó promoção,flutes,flauta slade transversal prata ccb promoção,flauta slade transversal prata ccb promoção,flaut slad transversal prat ccb promoçã
1576,flauta indígena tipo chaveiro,flutes,flauta indígena tipo chaveiro,flauta indígena tipo chaveiro,flaut indígen tip chaveir
8439,flauta transversal michael niquelada com pé em dó,flutes,flauta transversal michael niquelada com,flauta transversal michael niquelada,flaut transversal michael niquel
11202,flautim picollo eb 6 chaves antigo,flutes,flautim picollo chaves antigo,flautim picollo chaves antigo,flautim picoll chav antig
13029,flauta doce escolar para criança aprender toca...,flutes,flauta doce escolar para criança aprender toca...,flauta doce escolar criança aprender tocar yamaha,flaut doc escol crianc aprend toc yamah
6511,flauta transversal eagle fl05s,flutes,flauta transversal eagle fl05s,flauta transversal eagle fl05s,flaut transversal eagl fl05s
22506,flauta transversal tm51ni niquelada - csr,flutes,flauta transversal tm51ni niquelada csr,flauta transversal tm51ni niquelada csr,flaut transversal tm51ni niquel csr
23379,flauta artesanado profissional musico bambu t...,flutes,flauta artesanado profissional musico bambu t...,flauta artesanado profissional musico bambu tr...,flaut artesan profissional music bambu tradici...
9974,flauta transversal artley 8.0 prata maciça,flutes,flauta transversal artley prata maciça,flauta transversal artley prata maciça,flaut transversal artley prat macic
